In [4]:
import pandas as pd
from sodapy import Socrata
import os
from modules import data_management as dm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Download source files into your 'local' folder.

In [ ]:
dm.download_file_maps()
dm.download_files_raw_dengue()
dm.download_files_processed_dengue()
dm.download_file_population()
dm.download_file_year()
dm.download_file_departments()
dm.download_file_sstoi()
dm.download_file_tni()
dm.download_file_oni()
dm.download_file_meiv2()
dm.download_file_car()
dm.download_file_nta()
dm.download_rutinarias()

# Step by step download of climate data from two platforms...

## First, set the path to the folder where data will be stored

In [28]:
setup = "repo" #alternativo: local

try:
    if setup == "repo":
        data_folder = "/".join(dm.notebook_path().replace("\\","/").split("/")[:-1]) + "/local/data/"
        #data_folder = "./local/data/"
    else:
        data_folder = "/".join(dm.notebook_path().replace("\\","/").split("/")[:-1])  + "/data/"
        
except:
    print("Couldn't assess path automatically, setting up path locally...")
    data_folder = "D:/MyFiles/Research_Work/CW_One_Health/ALL_DENGUE/dengue-project/local/data/"
    
dm.make_folder_if_not_exists(data_folder)
data_folder

Couldn't assess path automatically, setting up path locally...


'D:/MyFiles/Research_Work/CW_One_Health/ALL_DENGUE/dengue-project/local/data/'

## Choose the city and retrieve all relevant data from params.csv

Amazonas
Antioquia
Arauca
Archipiélago de San Andres, Providencia y Santa Catalina
Atlantico
Bogotá
Bolivar
Boyacá
Caldas
Caquetá
Casanare
Cauca
Cesar
Chocó
Córdoba
Cundinamarca
Guainía
Guaviare
Huila
La Guajira
Magdalena
Meta
Nariño
Norte de Santander
Putumayo
Quindío
Risaralda
Santander
Sucre
Tolima
Valle del Cauca

In [102]:
df_cities = dm.read_cities_file()
df_cities

starting_date
stations_temp
stations_hum
stations_prec
stations_rad


,department,municipality,dl_folder,starting_date,stations_temp,stations_hum,stations_prec,stations_rad
0,Antioquia,Medellín,MEDELLIN_DATA,None,None,None,None,None
1,Magdalena,Santa Marta,SANTA_MARTA_DATA,None,None,None,None,None
2,Santander,Bucaramanga,SANTANDER_DATA,None,None,None,None,None
3,Atlántico,Barranquilla,BARRANQUILLA_SOLEDAD_DATA,None,None,None,None,None
4,Atlántico,Soledad,BARRANQUILLA_SOLEDAD_DATA,None,None,None,None,None
5,Norte de Santander,Cúcuta,CUCUTA_DATA,None,None,None,None,None
6,Valle del Cauca,Cali,CALI_DATA,None,None,None,None,None
7,Caquetá,Villavicencio,VILLAVICENCIO_DATA,None,None,None,None,None


In [111]:
row_id = 0
params = {}
params_to_get = ["department","municipality","dl_folder"]
for param in params_to_get:
    params[param] = df_cities.loc[row_id,param]
params

{'department': 'Antioquia',
 'municipality': 'Medellín',
 'dl_folder': 'MEDELLIN_DATA'}

In [112]:
dl_folder = data_folder + "src_per_city/" + params["dl_folder"]+"/"
dl_folder = dl_folder.replace("/", "\\")
dm.make_folder_if_not_exists(dl_folder)
dl_folder

'D:\\MyFiles\\Research_Work\\CW_One_Health\\ALL_DENGUE\\dengue-project\\local\\data\\src_per_city\\MEDELLIN_DATA\\'

## Download data from IDEAM's map using Selenium

The IDEAM API only allows downloads of these variables for up to 5 consecutive years, so we divide the relevant interval in 5 year segments.

Our relevant interval is 2006 to 2020. However, we also take years 2005 and 2021 for further review of the data.

In [84]:
variables = ["PRECIPITACION","HUM RELATIVA","TEMPERATURA","RAD SOLAR"]
date_intervals = [['01/01/2005','31/12/2009'],
                    ['01/01/2010','31/12/2014'],
                    ['01/01/2015','31/12/2019'],
                    ['01/01/2020','31/12/2021']
                ]
date_intervals

[['01/01/2005', '31/12/2009'],
 ['01/01/2010', '31/12/2014'],
 ['01/01/2015', '31/12/2019'],
 ['01/01/2020', '31/12/2021']]

In [ ]:
for interval in date_intervals:
    for variable in variables:
        res = dm.download_data_with_selenium(department, municipality, variable, interval, dl_folder)
        if res == True:
            print("Started download for:",department,"/",municipality,"/",variable,interval)
            try:
                os.rename(dl_folder+"datos.zip",dl_folder+"{}.zip".format(department+"-"+municipality+"-"+variable+"-"+interval[0].replace("/","-")+"-"+interval[1].replace("/","-")))
            except Exception as e:
                print(e)
                print("Error downloading, driver closed prematurely.")
        else:
            print("Couldn't find data for:",department,"/",municipality,"/",variable,interval)

In [ ]:
zipped_csvs = glob.glob(dl_folder+"*.zip")
files_count = 0
for zipped_csv in zipped_csvs:
    filename = zipped_csv.split("\\")[-1].split(".")[0]
    zipdata = zipfile.ZipFile(zipped_csv)
    zipdata.extract("excel.csv.csv", path = dl_folder)
    os.rename(dl_folder+"excel.csv.csv",dl_folder+"{}.csv".format(filename))
    files_count+=1
    zipdata.close()

# DOWNLOAD FROM DATOS ABIERTOS API


In [114]:
# Example authenticated client (needed for non-public datasets):
client = Socrata('www.datos.gov.co', "mc0EDr4vP5YMKFeiaAEHZUWX0")
Precipitacion = 's54a-sgyg'
Humedad = 'uext-mhny'
Temperatura = 'sbwg-7ju4'
Viento = 'sgfv-3yp8'

# PRECIPITATION DATA DOWNLOAD

In [118]:
prec_stations = dm.get_stations_from_municipality(client, Precipitacion, params["municipality"].upper())
prec_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConne

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0027015330,0240,APTO OLAYA HERRERA - TX GPRS,6.221,-75.589,Precipitacion,mm
1,0027015290,0240,PAJARITO - AUT,6.286333333,-75.61279167,Precipitacion,mm


In [123]:
codigos_estaciones = prec_stations["CodigoEstacion"].values
prec_time_series = []
for codigo in codigos_estaciones:
    precipitation_base_data = dm.get_time_series_from_station(client, Precipitacion,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000)
    precipitation_data = dm.group_variable(precipitation_base_data,'daily')
    prec_time_series.append(precipitation_data)
all_prec_series = pd.concat(prec_time_series,axis = 0).reset_index(drop=True)
all_prec_series = dm.API_to_IDEAM_series(all_prec_series)
all_prec_series.head(2)

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying


,CodigoEstacion,NombreEstacion,Latitud,Longitud,Altitud,Categoria,Entidad,AreaOperativa,Departamento,Municipio,...,FechaSuspension,IdParametro,Etiqueta,DescripcionSerie,Frecuencia,Fecha,Valor,Grado,Calificador,NivelAprobacion
0,27015330,APTO OLAYA HERRERA - TX GPRS,6.221,-75.589,,mm,,NECHÍ,ANTIOQUIA,MEDELLÍN,...,,,Precipitacion,,,2016-12-31T00:00:00.000,0.0,,,
1,27015330,APTO OLAYA HERRERA - TX GPRS,6.221,-75.589,,mm,,NECHÍ,ANTIOQUIA,MEDELLÍN,...,,,Precipitacion,,,2017-01-01T00:00:00.000,0.0,,,


In [128]:
all_prec_series.to_csv(dl_folder+"API_{}_precipitacion.csv".format(params["municipality"]), index = False)

# TEMPERATURE DATA DOWNLOAD

In [130]:
temp_stations = dm.get_stations_from_municipality(client, Temperatura, params["municipality"].upper())
temp_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying


,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0027015330,0068,APTO OLAYA HERRERA - TX GPRS,6.221,-75.589,Temp Aire 2 m,°C
1,0027015300,0068,VILLANUEVA,6.253,-75.55,Temp Aire 2 m,°C
2,0027015290,0068,PAJARITO - AUT,6.286333333,-75.61279167,Temp Aire 2 m,°C


In [131]:
codigos_estaciones = temp_stations["CodigoEstacion"].values
temp_time_series = []
for codigo in codigos_estaciones:
    temperature_data = dm.group_variable(dm.get_time_series_from_station(client, Temperatura,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000), 'hourly')
    temp_time_series.append(temperature_data)
all_temp_series = pd.concat(temp_time_series,axis = 0).reset_index(drop=True)
all_temp_series = dm.API_to_IDEAM_series(all_temp_series)
all_temp_series.to_csv(dl_folder+"/API_{}_temperature.csv".format(params["municipality"]), index = False)

# HUMEDAD DATA DOWNLOAD

In [133]:
hum_stations = dm.get_stations_from_municipality(client, Humedad, params["municipality"].upper())
hum_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying


,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0027015290,0027,PAJARITO - AUT,6.286333333,-75.61279167,Humedad del aire 2 mt,%
1,0027015300,0027,VILLANUEVA,6.253,-75.55,Humedad del aire 2 mt,%
2,0027015330,0027,APTO OLAYA HERRERA - TX GPRS,6.221,-75.589,Humedad del aire 2 mt,%


In [134]:
codigos_estaciones = hum_stations["CodigoEstacion"].values
hum_time_series = []
for codigo in codigos_estaciones:
    humedad_data = dm.group_variable(dm.get_time_series_from_station(client, Humedad,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000))
    hum_time_series.append(humedad_data)
hum_time_series = pd.concat(hum_time_series,axis = 0).reset_index(drop=True)
hum_time_series = dm.API_to_IDEAM_series(hum_time_series)
hum_time_series.to_csv(dl_folder+"/API_{}_humidity.csv".format(params["municipality"]), index = False)

# WIND DATA DOWNLOAD

In [136]:
win_stations = dm.get_stations_from_municipality(client, Viento, params["municipality"].upper())
win_stations

HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConnectionPool(host='www.datos.gov.co', port=443): Read timed out. (read timeout=10) retrying
HTTPSConne

,CodigoEstacion,CodigoSensor,NombreEstacion,Latitud,Longitud,DescripcionSensor,UnidadMedida
0,0027015290,0103,PAJARITO - AUT,6.286333333,-75.61279167,Velocidad Viento (10 min),m/s
1,0027015330,0103,APTO OLAYA HERRERA - TX GPRS,6.221,-75.589,Velocidad Viento (10 min),m/s
2,0027015300,0103,VILLANUEVA,6.253,-75.55,Velocidad Viento (10 min),m/s


In [137]:
codigos_estaciones = win_stations["CodigoEstacion"].values
win_time_series = []
for codigo in codigos_estaciones:
    wind_data = dm.group_variable(dm.get_time_series_from_station(client, Viento,  params["municipality"].upper(), codigoestacion=codigo, limit = 1000000))
    win_time_series.append(wind_data)
win_time_series = pd.concat(win_time_series,axis = 0).reset_index(drop=True)
win_time_series = dm.API_to_IDEAM_series(win_time_series)
win_time_series.to_csv(dl_folder+"/zAPI_{}_wind.csv".format(municipality), index = False)

In [ ]:
#Downloads from IDEAM with Selenium
variables = ["VEL VIENTO"]
#variables = ["RAD SOLAR"]
#variables = ["RAD SOLAR"]
date_intervals = [ [["01/01/{}".format(year),"31/01/{}".format(year)],
                    ["01/02/{}".format(year),"29/02/{}".format(year)],
                    ["01/03/{}".format(year),"31/03/{}".format(year)],
                    ["01/04/{}".format(year),"30/04/{}".format(year)],
                    ["01/05/{}".format(year),"31/05/{}".format(year)],
                    ["01/06/{}".format(year),"30/06/{}".format(year)],
                    ["01/07/{}".format(year),"31/07/{}".format(year)],
                    ["01/08/{}".format(year),"31/08/{}".format(year)],
                    ["01/09/{}".format(year),"30/09/{}".format(year)],
                    ["01/10/{}".format(year),"31/10/{}".format(year)],
                    ["01/11/{}".format(year),"30/11/{}".format(year)],
                    ["01/12/{}".format(year),"31/12/{}".format(year)]] 
                  for year in range(2007,2021)]
date_intervals = [di for di_tup in date_intervals for di in di_tup]